# Рынок заведений общественного питания Москвы

Описание проекта  

Инвесторы из фонда «Shut Up and Take My Money» решили попробовать себя в новой области и открыть заведение общественного питания в Москве. Заказчики ещё не знают, что это будет за место: кафе, ресторан, пиццерия, паб или бар, — и какими будут расположение, меню и цены.
Необходимо подготовить исследование рынка Москвы, найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего инвесторам места.  




Описание данных  

Файл moscow_places.csv:  

name — название заведения;  

address — адрес заведения;  

category — категория заведения, например «кафе», «пиццерия» или «кофейня»;  

hours — информация о днях и часах работы;  

lat — широта географической точки, в которой находится заведение;  

lng — долгота географической точки, в которой находится заведение;  

rating — рейтинг заведения по оценкам пользователей в Яндекс Картах (высшая оценка — 5.0);  

price — категория цен в заведении, например «средние», «ниже среднего», «выше среднего» и так далее;  

avg_bill — строка, которая хранит среднюю стоимость заказа в виде диапазона, например:  

«Средний счёт: 1000–1500 ₽»;  

«Цена чашки капучино: 130–220 ₽»;  

«Цена бокала пива: 400–600 ₽».  

middle_avg_bill — число с оценкой среднего чека, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Средний счёт»:  

Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений.
Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число.
Если значения нет или оно не начинается с подстроки «Средний счёт», то в столбец ничего не войдёт.  

middle_coffee_cup — число с оценкой одной чашки капучино, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Цена одной чашки капучино»:
Если в строке указан ценовой диапазон из двух значений, в столбец войдёт медиана этих двух значений.
Если в строке указано одно число — цена без диапазона, то в столбец войдёт это число.
Если значения нет или оно не начинается с подстроки «Цена одной чашки капучино», то в столбец ничего не войдёт.  

chain — число, выраженное 0 или 1, которое показывает, является ли заведение сетевым (для маленьких сетей могут встречаться ошибки):
0 — заведение не является сетевым
1 — заведение является сетевым  

district — административный район, в котором находится заведение, например Центральный административный округ;  

seats — количество посадочных мест.

План:
- загрузка данных и изучение общей информации
- предобработка данных
- анализ данных
- детализация исследования: открытие кофейни
- подготовка презентации

## Шаг: загрузка и изучение общей информации

In [1]:
#Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import math as mth
from scipy import stats as st
import re
import plotly.express as px
from plotly import graph_objects as go
# подключаем модуль для работы с JSON-форматом
import json
# импортируем карту и хороплет
from folium import Map, Choropleth
# импортируем карту и маркер
from folium import Map, Marker
# импортируем кластер
from folium.plugins import MarkerCluster

ModuleNotFoundError: No module named 'folium'

In [ ]:
data = pd.read_csv('/datasets/moscow_places.csv')
data.head(20)

In [ ]:
data.info()

## Шаг: предобработка данных

In [ ]:
#приведем к нижнему регистру
for col in ['name', 'category', 'address', 'hours', 'price', 'avg_bill']:
    data[col] = data[col].str.lower()

In [ ]:
# Проверим наличие дубликатов
data.duplicated().sum()

In [ ]:
data.duplicated(['name', 'address']).sum()

In [ ]:
data.drop_duplicates(subset=['name', 'address'], keep=False)

In [ ]:
data.duplicated(['name', 'address']).sum()

In [ ]:
#Код ревьюера
data = data.drop_duplicates(subset=['name', 'address'], keep='last')

In [ ]:
#Код ревьюера
data[['name', 'address']].duplicated().sum()

In [ ]:
#поменяем тип данных
data['seats'] = data['seats'].astype('float')

In [ ]:
#проверка пропусков
data.isna().sum()

In [ ]:
#оценим доли пропусков
data.isna().sum() / data.shape[0]

In [ ]:
data['seats'].describe()

In [ ]:
#Отрисуем ящики с усами 
data.boxplot(column=['seats'], grid= False , color='black')

In [ ]:
data['seats'].hist(bins=100,range=(0, 1500))
plt.show()

Тут мы видим странный выброс данных, максимальное количество мест сильно отличается, необходимо скорректировать 

In [ ]:
data['seats'].describe()

In [ ]:
def street(address):
    address = address.strip().lower()
    address = re.sub(r'\s*,\s*', ',', address)
    address = re.split(',|\.', address)
    return address[1]

In [ ]:
def is_24_7(time):
    if time != time:
        return np.nan
    elif time == 'ежедневно, круглосуточно':
        return True
    else:
        return False

In [ ]:
#добавим столбец с названиями улиц из столбца с адресом
data['street'] = data['address'].apply(street)

In [ ]:
#добавим столбец с обозначением, что заведение работает ежедневно и круглосуточно (24/7)
data['is_24/7'] = data['hours'].apply(is_24_7)

In [ ]:
data.head()

Вывод:
- привели данные к нижнему регистру 
- дубликатов найдено не было, за исключением тех, что попали в оба столбца 'name'и 'address'
- восстановить пропуски причины нет
- поменяли тип данных seats	
- ошибока данных была обнаружена в столбце seats, это выбросы, которые мы с корректировали
- были добавлены столбцы с названиями улиц и с маркером работы 24/7

## Шаг: анализ данных

**Какие категории заведений представлены в данных? Исследуйте количество объектов общественного питания по категориям: рестораны, кофейни, пиццерии, бары и так далее. Постройте визуализации. Ответьте на вопрос о распределении заведений по категориям.**

In [ ]:
# готовим данные для графика
df_category = pd.DataFrame(data['category'].value_counts()).reset_index()
# строим диаграмму с сегментами
fig = go.Figure(data=[go.Pie(labels=df_category['index'], # указываем значения, которые появятся на метках сегментов
                             values=df_category['category'], # указываем данные, которые отобразятся на графике
                             pull = [0.1, 0])]) # добавляем аргумент, который выделит сегмент-лидер на графике
fig.update_layout(title='Количество объектов общественного питания по категориям', # указываем заголовок графика
                  width=800, # указываем размеры графика
                  height=600,
                  annotations=[dict(x=1.12, # вручную настраиваем аннотацию легенды
                                    y=1.05,
                                    text='Категории',
                                    showarrow=False)])
fig.show() # выводим график

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Распределение заведений по категориям',fontsize=15)
sns.barplot(data=df_category, x='category', y='index')
plt.grid(True) #отразим сетку для удобства
plt.xlabel('Количество',fontsize=12)
plt.ylabel('Категории',fontsize=12)

In [ ]:
# Вывожу таблицу на экран
df_category.columns = ['Тип объекта', 'Количество']
display(df_category) 

*Вывод:*  

Наибольшую долю рынка общественного питания в городе Москва занимают кафе - 28.3%, далее идут рестораны (24.3%), кофейни (16.8%), бар, паб (9.1%), пиццерия (7.53%), быстрое питание (7.17%), столовая (3.75%), булочная (3.05%). 

**Исследуйте количество посадочных мест в местах по категориям: рестораны, кофейни, пиццерии, бары и так далее. Постройте визуализации. Проанализируйте результаты и сделайте выводы.**

In [ ]:
data.groupby(by='category', as_index=False).agg(
    min=('seats', 'min'),
    max=('seats', 'max'),
    mean=('seats', 'mean'),
    median=('seats', 'median')
)

In [ ]:
mean_number = data.groupby('category').agg({'seats':'mean'}).sort_values(by='seats', ascending=False).reset_index().round()
fig = go.Figure(data=[go.Bar(x = mean_number['category'],
                             y = mean_number['seats'],
                             text= mean_number['seats'],
                             textposition = 'outside', name='category'
                             )])
fig.update_layout(title_text='Среднее кол-во посадочных мест по виду заведений')
fig.show()

Вывод: В среднем наибольшее количество посадочных мест характерно для баров, пабов и ресторанов - 125 и 122 мест соответственно. Меньше всего посадочных мест в булочных - 89. Это объясняется спецификой работы данных заведений(баров, пабов и ресторанов): определенный временной промежуток довольно большой и чтобы удовлетворить спрос требуется начилие большого количества посадочных мест. А если говорить о пиццериях и булочных, то тут характерна модель "еда на вынос", поэтому и большого количество посадочных мест тут не требуется.

**Рассмотрите и изобразите соотношение сетевых и несетевых заведений в датасете. Каких заведений больше?
Какие категории заведений чаще являются сетевыми? Исследуйте данные и ответьте на вопрос графиком.**

In [ ]:
fig = go.Figure(go.Pie(
    labels=['Несетевые', 'Сетевые'],
    values = data['chain'].value_counts().to_list(), title='Cоотношение сетевых и несетевых заведений'))
fig.update_traces(textposition='inside', textinfo='percent+label', pull=[0,0.2])
fig.show()

Вывод: Несетевых заведений значительно больше, чем сетевых - 61.9% к 38.1% соответственно

In [ ]:
df_chain = data.pivot_table(index=['chain', 'category'], values='seats', aggfunc='count')
df_chain

In [ ]:
# строим гистограммы
fig = px.histogram(data, # загружаем данные
                   x='category', # указываем столбец с данными для оси X
                   color='chain', # обозначаем категорию для разделения цветом
                   range_x=[0, 10], # ограничиваем ось X
                   title='Распределение сетевых и несетевых заведений в датасете', # указываем заголовок
                   nbins=1000, # назначаем число корзин
                   barmode='overlay') # выбираем «полупрозрачный» тип отображения столбцов
fig.update_xaxes(title_text='Категория') # подпись для оси X
fig.update_yaxes(title_text='Частота') # подпись для оси Y
fig.show() # выводим график

Вывод: Чаще всего сетевыми заведениями являются такие категрии как кофейня, пиццерия, булочная

In [ ]:
chain_categories = data.groupby(['category', 'chain']).agg(count=('category', 'count')).sort_values('category').reset_index()
chain_categories['percentage'] = data.groupby(['category', 'chain']).size().groupby(level=0).apply(lambda x: 100 * x / float(x.sum())).values
chain_categories['chain'] = chain_categories['chain'].apply(lambda x: 'сетевые' if x == 1 else 'несетевые')
chain_categories.columns = ['category', 'chain', 'counts', 'percentage']
chain_categories

In [ ]:
# назначаем размер графика
plt.figure(figsize=(10, 4))
# строим столбчатый график средствами seaborn
sns.barplot(x='percentage', y='category', data=chain_categories, hue='chain')

plt.title('График доли сетевых заведений в разрезе категорий от общего количества заведений')
plt.xlabel('Доля')
plt.ylabel('Категория заведения')
# поворачиваем подписи значений по оси X на 45 градусов
plt.xticks(rotation=45)
# выбираем положение легенды и указываем размер шрифта
plt.legend(loc='lower right', fontsize=10)
# добавляем сетку
plt.grid()
# отображаем график на экране
plt.show()

<div class="alert alert-success" style="border-radius: 10px; box-shadow: 2px 2px 2px; border: 1px solid; padding: 10px ">
<b>Комментарий ревьюера v.3</b> 
    
👍 
Можно еще вот так :)
</div>

In [ ]:
# #Код ревьюера
fig = px.bar(chain_categories, 
             x='counts',
             y='category', 
             text = chain_categories['percentage'].apply(lambda x: '{0:1.2f}%'.format(x)),              
             color='chain',
             category_orders={"chain": ["сетевые", "несетевые"]},
            )
fig.update_layout(title='Соотношение сетевых заведений',
                   xaxis_title='Количество заведений',
                   yaxis_title='Название категорий',
                   yaxis={'categoryorder': 'total ascending'}
                 )
fig.show()

**Сгруппируйте данные по названиям заведений и найдите топ-15 популярных сетей в Москве. Под популярностью понимается количество заведений этой сети в регионе. Постройте подходящую для такой информации визуализацию. Знакомы ли вам эти сети? Есть ли какой-то признак, который их объединяет? К какой категории заведений они относятся?**

In [ ]:
# сгруппируем по названиям заведений
df_top_name = data.groupby('name')[['name']].count()
# переименуем столбец
df_top_name.columns = ['name_count']
# отсортируем и оставим пятнадцать лидеров
df_top_name = df_top_name.reset_index().sort_values(by='name_count', ascending=False).head(15)
display(df_top_name)

In [ ]:
data_cat = data[['name', 'category']]
top_15_cat = df_top_name.merge(data_cat, how='left', on='name')
top_15_cat.pivot_table(index=['name', 'category']).sort_values(by='name_count', ascending=False)

In [ ]:
# строим столбчатую диаграмму 
fig = px.bar(df_top_name.sort_values(by='name_count', ascending=True), # загружаем данные и заново их сортируем
             x='name_count', # указываем столбец с данными для оси X
             y='name', # указываем столбец с данными для оси Y
             text='name_count' # добавляем аргумент, который отобразит текст с информацией
                                # о количестве объявлений внутри столбца графика
            )
# оформляем график
fig.update_layout(title='ТОП-15 популярных сетей в Москве',
                   xaxis_title='Количество',
                   yaxis_title='Сеть')
fig.show() # выводим график

Признак, которым объеденены популярные сети это категория заведения, а именно - кафе

In [ ]:
chain_top_15 = data[data['chain'] == 1]
chain_top_15 = chain_top_15.groupby(['name', 'category']).agg({'name' : 'count'})
chain_top_15.columns = ['count']
chain_top_15 = chain_top_15.sort_values('count', ascending = False).reset_index().head(15)
chain_top_15.head(15)

In [ ]:
# назначаем размер графика
plt.figure(figsize=(20, 8))
# строим столбчатый график средствами seaborn
sns.barplot(x='count', y='name', data=chain_top_15, hue='category')

plt.title('График сетевых заведений по категориям')
plt.xlabel('Количество')
plt.ylabel('Название сети')
# поворачиваем подписи значений по оси X на 45 градусов
plt.xticks(rotation=45)
# выбираем положение легенды и указываем размер шрифта
plt.legend(loc='lower right', fontsize=10)
# добавляем сетку
plt.grid()
# отображаем график на экране
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot(x='category', y='count', data=chain_top_15, hue='name')
plt.title('График сетевых заведений по категориям')
plt.xlabel('Количество')
plt.ylabel('Категория заведения')
# поворачиваем подписи значений по оси X на 45 градусов
plt.xticks(rotation=45)
# выбираем положение легенды и указываем размер шрифта
plt.legend(loc='upper right', fontsize=10)
# добавляем сетку
plt.grid()
# отображаем график на экране
plt.show()

In [ ]:
chain_top_15_category =  (chain_top_15.groupby('category', as_index=False)
                              .agg(count_object=('name', 'count')))

chain_top_15_category['ratio'] = (chain_top_15_category['count_object'] / \
                                  chain_top_15_category['count_object'].sum()*100).round().astype('int')
chain_top_15_category = chain_top_15_category.sort_values(by='ratio', ascending=False)
chain_top_15_category

In [ ]:
# назначаем размер графика
plt.figure(figsize=(10, 4))
# строим столбчатый график средствами seaborn
sns.barplot(x='ratio', y='category', data=chain_top_15_category)

plt.title('График доли сетевых заведений в разрезе категорий от общего количества заведений')
plt.xlabel('Доля')
plt.ylabel('Категория заведения')

# добавляем сетку
plt.grid()
# отображаем график на экране
plt.show()

Вывод: доля сетевых заведений, являющихся топовыми, привалирует в категории кофейня, далее кафе, ресторан, пиццерия и булочная

Какие административные районы Москвы присутствуют в датасете? Отобразите общее количество заведений и количество заведений каждой категории по районам. Попробуйте проиллюстрировать эту информацию одним графиком.

In [ ]:
data['district'].unique()

In [ ]:
district = data['district'].unique()
print(district)

In [ ]:
plt.figure(figsize=(20, 10))
sns.countplot(x='district', hue='category', data=data)
plt.xticks(rotation=45)
plt.title('Распределение категорий по районам')
plt.show()

Вывод:  лидирующую позицию по количеству заведени всех категорий занимает Центральный административный округ(ЦАО), также мы можем отметить, что в этом округе больше всего ресторанов, тогда как в других округах лидирует категория кафе. Это можно объяснить тем, что в центральной части города обычно собраны самые высокие категории заведений общественного питания. 

**Визуализируйте распределение средних рейтингов по категориям заведений. Сильно ли различаются усреднённые рейтинги в разных типах общепита?**


In [ ]:
# посчитаем медианный рейтинг
rating_category = data.groupby('category', as_index=False)['rating'].agg('mean')
rating_category

In [ ]:
# строим столбчатую диаграмму 
fig = px.bar(rating_category.sort_values(by='rating', ascending=True), # загружаем данные и заново их сортируем
             x='rating', # указываем столбец с данными для оси X
             y='category', # указываем столбец с данными для оси Y
             text='rating' # добавляем аргумент, который отобразит текст с информацией
                                # о количестве объявлений внутри столбца графика
            )
# оформляем график
fig.update_layout(title='Распределение средних рейтингов по категориям заведений',
                   xaxis_title='Рейтинг',
                   yaxis_title='Категория')
fig.show() # выводим график

Вывод: усреднённые рейтинги в разных типах общепита не очень различаются 4.05 и 4.38 - разброс всего то 0.33

**Постройте фоновую картограмму (хороплет) со средним рейтингом заведений каждого района. Границы районов Москвы, которые встречаются в датасете, хранятся в файле admin_level_geomap.geojson (скачать файл для локальной работы).**

In [ ]:
rating_df = data.groupby('district', as_index=False)['rating'].agg('median')
rating_df

In [ ]:
# подключаем модуль для работы с JSON-форматом
import json

# читаем файл и сохраняем в переменной
with open('/datasets/admin_level_geomap.geojson', 'r') as f:
    geo_json = json.load(f)

print(json.dumps(geo_json, indent=2, ensure_ascii=False, sort_keys=True))

In [ ]:
# загружаем JSON-файл с границами округов Москвы
state_geo = '/datasets/admin_level_geomap.geojson'
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=rating_df,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='RdPu',
    fill_opacity=0.8,
    legend_name='Медианный рейтинг заведений по районам',
).add_to(m)

# выводим карту
m

**Отобразите все заведения датасета на карте с помощью кластеров средствами библиотеки folium.**

In [ ]:
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
data.apply(create_clusters, axis=1)

# выводим карту
m

**Найдите топ-15 улиц по количеству заведений. Постройте график распределения количества заведений и их категорий по этим улицам. Попробуйте проиллюстрировать эту информацию одним графиком.**

In [ ]:
#топ 15 улиц
top_street = data.pivot_table(index = 'street', values = 'name', aggfunc = 'count')
top_15 = top_street.sort_values(by='name', ascending=False).head(15).reset_index()
top_15['category'] = data['category']

In [ ]:
top_15

In [ ]:
plt.figure(figsize=(20, 10))
sns.barplot(x='street', y='name', data=top_15, hue='category')
plt.title('График топ-15 улиц по количеству заведений по категориям')
plt.xlabel('Улицы')
plt.ylabel('Количество')
# поворачиваем подписи значений по оси X на 45 градусов
plt.xticks(rotation=45)
# выбираем положение легенды и указываем размер шрифта
plt.legend(loc='upper right', fontsize=10)
# добавляем сетку
plt.grid()
# отображаем график на экране
plt.show()

Вывод: больше всего заведений на пр. Мира, ул. Профсоюзной. пр. Вернадского и Ленинский пр., обусловлено это тем что эти улицы очень длинные. А самые популярные категории заведений кафе и ресторан.

In [ ]:
street.plot(kind='bar',figsize=(12, 8))


**Найдите улицы, на которых находится только один объект общепита. Что можно сказать об этих заведениях?**

In [ ]:
one_cafe = data.groupby('street').agg({'name':'count'}).reset_index().query('name == 1')
one_cafe

Вывод: Можно предположить,  что  улицами с одним объектом общепита являются непопулярные улицы с низким трафиком.

**Значения средних чеков заведений хранятся в столбце middle_avg_bill. Эти числа показывают примерную стоимость заказа в рублях, которая чаще всего выражена диапазоном. Посчитайте медиану этого столбца для каждого района. Используйте это значение в качестве ценового индикатора района. Постройте фоновую картограмму (хороплет) с полученными значениями для каждого района. Проанализируйте цены в центральном административном округе и других. Как удалённость от центра влияет на цены в заведениях?**

In [ ]:

# Подготовим таблицу медианного значения стоимости заказов в разрезе округов
district_map_price = data.groupby('district', as_index=False)['middle_avg_bill'].agg('median').round(1)

In [ ]:
# Создаем горизонтальный столбчатый график
fig = px.bar(district_map_price, x='middle_avg_bill', y='district', orientation='h',
             color='middle_avg_bill', text='middle_avg_bill')

fig.update_layout(title='Медиана среднего чека в рублях по округам',
                  xaxis_title='Средний чек в рублях. Медиана для адм. округа',
                  yaxis_title='Адм. округа',
                  showlegend=False)

fig.update_yaxes(categoryorder='total ascending') 

fig.show()

In [ ]:
# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=district_map_price,
    columns=['district', 'middle_avg_bill'],
    key_on='feature.name',
    fill_color='RdPu',
    fill_opacity=0.5,
    legend_name='Чек заведений по районам',
).add_to(m)

# выводим карту
m

Вывод: районы с самым высоким средним чеком - ЦАО и ЗАО. Удаленность от центра влияет на средний чек, чем дальше, тем чек ниже 

**Вывод по анализу данных:**  

Мы выяснили, аибольшую долю рынка общественного питания в городе Москва занимают кафе - 28.3%, далее идут рестораны (24.3%), кофейни (16.8%), бар, паб (9.1%), пиццерия (7.53%), быстрое питание (7.17%), столовая (3.75%), булочная (3.05%). 

При этом несетевых заведений значительно больше, чем сетевых - 61.9% к 38.1% соответственно.
Доля сетевых заведений, являющихся топовыми, привалирует в категории кофейня, далее кафе, ресторан, пиццерия и булочная

В среднем наибольшее количество посадочных мест характерно для баров, пабов и ресторанов - 125 и 122 мест соответственно. Меньше всего посадочных мест в булочных - 89. Это объясняется спецификой работы данных заведений(баров, пабов и ресторанов): определенный временной промежуток довольно большой и чтобы удовлетворить спрос требуется начилие большого количества посадочных мест. А если говорить о пиццериях и булочных, то тут характерна модель "еда на вынос", поэтому и большого количество посадочных мест тут не требуется.

Лидирующую позицию по количеству заведени всех категорий занимает Центральный административный округ(ЦАО), также мы можем отметить, что в этом округе больше всего ресторанов, тогда как в других округах лидирует категория кафе. Это можно объяснить тем, что в центральной части города обычно собраны самые высокие категории заведений общественного питания. 

Усреднённые рейтинги в разных типах общепита не очень различаются 4.05 и 4.38 - разброс всего то 0.33

Улицами с одним объектом общепита являются непопулярные улицы с низким трафиком.

Больше всего заведений на пр. Мира, ул. Профсоюзной. пр. Вернадского и Ленинский пр., обусловлено это тем что эти улицы очень длинные. А самые популярные категории заведений кафе и ресторан.

Районы с самым высоким средним чеком - ЦАО и ЗАО. Удаленность от центра влияет на средний чек, чем дальше, тем чек ниже 



## Шаг: детализация исследования: открытие кофейни

Шаг 4. Детализируем исследование: открытие кофейни
Основателям фонда «Shut Up and Take My Money» не даёт покоя успех сериала «Друзья». Их мечта — открыть такую же крутую и доступную, как «Central Perk», кофейню в Москве. Будем считать, что заказчики не боятся конкуренции в этой сфере, ведь кофеен в больших городах уже достаточно. Попробуйте определить, осуществима ли мечта клиентов.  

Ответьте на следующие вопросы:
Сколько всего кофеен в датасете? В каких районах их больше всего, каковы особенности их расположения?
Есть ли круглосуточные кофейни?
Какие у кофеен рейтинги? Как они распределяются по районам?
На какую стоимость чашки капучино стоит ориентироваться при открытии и почему?
По желанию вы можете расширить список вопросов для исследования, добавив собственные.
Постройте визуализации. Попробуйте дать рекомендацию для открытия нового заведения. Это творческое задание: здесь нет правильного или неправильного ответа, но ваше решение должно быть чем-то обосновано. Объяснить свою рекомендацию можно текстом с описанием или маркерами на географической карте.

In [ ]:
# создадим отдельную таблицу только с кофейнями
coffee_data = data[data['category'] == 'кофейня']

In [ ]:
coffee_house = data.query('category == "кофейня"')
print('Всего кофеен в датасете', len(coffee_house))



In [ ]:
cofe_cup = coffee_data.groupby('district', as_index=False)['name'].agg('count')

In [ ]:
state_geo = '/datasets/admin_level_geomap.geojson'
moscow_lat, moscow_lng = 55.751244, 37.618423
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
Choropleth(
    geo_data=state_geo,
    data=cofe_cup,
    columns=['district', 'name'],
    key_on='feature.name',
    fill_color='RdPu',
    fill_opacity=0.8,
    legend_name='Количество кофеен по районам.',
).add_to(m)
m

In [ ]:
okrug = coffee_data.groupby('district', as_index=False)['is_24/7'].agg('sum')

In [ ]:
state_geo = '/datasets/admin_level_geomap.geojson'
moscow_lat, moscow_lng = 55.751244, 37.618423
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
Choropleth(
    geo_data=state_geo,
    data=okrug,
    columns=['district', 'is_24/7'],
    key_on='feature.name',
    fill_color='RdPu',
    fill_opacity=0.8,
    legend_name='Количество круглосуточных  по кофеен.',
).add_to(m)
m

In [ ]:
site = coffee_data.groupby('district', as_index=False)['seats'].agg('median')

In [ ]:
state_geo = '/datasets/admin_level_geomap.geojson'
moscow_lat, moscow_lng = 55.751244, 37.618423
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
Choropleth(
    geo_data=state_geo,
    data=site,
    columns=['district', 'seats'],
    key_on='feature.name',
    fill_color='RdPu',
    fill_opacity=0.8,
    legend_name='Количество посадочных мест.',
).add_to(m)
m

In [ ]:
one_cup = coffee_data.groupby('district', as_index=False)['middle_coffee_cup'].agg('median')

In [ ]:
state_geo = '/datasets/admin_level_geomap.geojson'
moscow_lat, moscow_lng = 55.751244, 37.618423
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
Choropleth(
    geo_data=state_geo,
    data=one_cup,
    columns=['district', 'middle_coffee_cup'],
    key_on='feature.name',
    fill_color='RdPu',
    fill_opacity=0.8,
    legend_name='Медианная цена чашки кофе.',
).add_to(m)
m

In [ ]:
data['rating'].describe()

In [ ]:
# назначаем классический стиль для графика с помощью matplotlib
plt.style.use('classic')
# строим гистограмму для столбца total_income, добавив график кривой KDE
sns.histplot(x='rating', data=data, kde=True)
# ограничиваем ось X для наглядности
plt.xlim(1, 5)
# указываем заголовок графика и подписи осей средствами matplotlib
plt.title('Распределение рейтингов')
# отображаем график на экране
plt.show()

In [ ]:
# используем стиль dark из библиотеки seaborn
sns.set_style('dark')
# назначаем размер графика
plt.figure(figsize=(10, 5))
# строим столбчатый график средствами seaborn
sns.barplot(x='category', y='rating', data=data)
# формируем заголовок графика и подписи осей средствами matplotlib
plt.title('График рейтингов зависимости от категории')
plt.xlabel('Категория')
plt.ylabel('Рейтинг')
# поворачиваем подписи значений по оси X на 45 градусов
plt.xticks(rotation=45)

# добавляем сетку
plt.grid()
# отображаем график на экране
plt.show()

Рекомендации:

На рынке более популярны кафе, потому именно такой тип заведения и рекомендовать со средним количеством посадочных мест на 40 персон. Касаемо района и улиц, то предполагаем что объект лучше размещать на одной из топ-15 улиц, ведь не спроста на этих улицах наибольшее количество объектов. Учитывая, что мечта инвесторов — открыть такую же доступную, как «Central Perk», кофейню в Москве, то  стоимость чашки капучино может быть -  140 - 160р. 



Презентация: <https://disk.yandex.ru/d/97hwLCFvM2U4ZQ>